In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)

In [3]:
trade = pd.read_csv("../data/trade-information.csv")
ir = pd.read_csv("../data/trade-price-ir-vegas.csv")
swap = pd.read_csv("../data/market-data-swap-rates.csv")
swaption = pd.read_csv("../data/market-data-swaption-vols.csv")

In [4]:
trade['bound width'] = trade['upper_bound'] - trade['lower_bound'] 

trade['month_pay_frequency'] = trade['pay_frequency'].str.removesuffix("M") # remove M from string
trade['month_pay_frequency'] = trade['month_pay_frequency'].astype(int) # convert to int


trade['year_maturity'] = trade['maturity'].str.removesuffix("Y") # remove Y from string
trade['year_maturity'] = trade['year_maturity'].astype(int)  # convert to int


In [5]:
trade

,trade name,underlying,pay_frequency,maturity,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,dummyTrade1,USD: CMS:2Y,6M,5Y,0.0042,0.0379,0.0337,6,5
1,dummyTrade2,USD: CMS:2Y,4M,5Y,0.0042,0.0379,0.0337,4,5
2,dummyTrade3,USD: CMS:5Y,6M,5Y,0.0044,0.0397,0.0353,6,5
3,dummyTrade4,USD: CMS:5Y,4M,5Y,0.0044,0.0397,0.0353,4,5
4,dummyTrade5,USD: CMS:10Y,4M,5Y,0.0046,0.0417,0.0371,4,5
5,dummyTrade6,USD: CMS:10Y,3M,5Y,0.0046,0.0417,0.0371,3,5
6,dummyTrade7,USD: CMS:2Y,6M,10Y,0.0042,0.0379,0.0337,6,10
7,dummyTrade8,USD: CMS:2Y,4M,10Y,0.0042,0.0379,0.0337,4,10
8,dummyTrade9,USD: CMS:5Y,6M,10Y,0.0044,0.0397,0.0353,6,10
9,dummyTrade10,USD: CMS:5Y,4M,10Y,0.0044,0.0397,0.0353,4,10


In [6]:
swap

,Date,Start Date,Tenor,Swap Rate
0,13/1/2021,13/1/2022,10y,1.081333
1,13/1/2021,13/1/2022,15y,1.265960
2,13/1/2021,13/1/2022,18M,0.221625
3,13/1/2021,13/1/2022,1y,0.195474
4,13/1/2021,13/1/2022,2y,0.279370
...,...,...,...,...
63175,12/1/2024,12/1/2039,2y,3.424505
63176,12/1/2024,12/1/2039,3y,3.393758
63177,12/1/2024,12/1/2039,4y,3.358897
63178,12/1/2024,12/1/2039,5y,3.319531


In [7]:
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2/9/2022,dummyTrade1,USD,-100,-227907.0988,1y,4/9/2023,10y,1.962246
1,2/9/2022,dummyTrade1,USD,-50,-222208.4010,1y,4/9/2023,10y,-3.812341
2,2/9/2022,dummyTrade1,USD,-25,-218960.9280,1y,4/9/2023,10y,4.471006
3,2/9/2022,dummyTrade1,USD,-10,-216872.4301,1y,4/9/2023,10y,4.333398
4,2/9/2022,dummyTrade1,USD,-5,-216146.3103,1y,4/9/2023,10y,5.679687
...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,-197572.2089,15y,2/2/2038,10y,0.104718
1048571,2/2/2023,dummyTrade6,USD,100,-186933.5679,15y,2/2/2038,10y,0.238781
1048572,2/2/2023,dummyTrade7,USD,-100,-680981.2765,15y,2/2/2038,10y,2.552405
1048573,2/2/2023,dummyTrade7,USD,-50,-656248.2494,15y,2/2/2038,10y,2.687706


In [8]:
# dictionary storing for tenor
dummy2YCMS = {'dummyTrade1': 2, 'dummyTrade2': 2, 'dummyTrade7': 2, 'dummyTrade8': 2}
dummy5YCMS = {'dummyTrade3': 5, 'dummyTrade4': 5, 'dummyTrade9': 5, 'dummyTrade10': 5}
dummy10YCMS = {'dummyTrade5': 10, 'dummyTrade6': 10, 'dummyTrade11': 10, 'dummyTrade12': 10}
YtoInt = {2:"2y", 5: "5y", 10: "10y"}

dummyTenorCMS = {**dummy2YCMS, **dummy5YCMS, **dummy10YCMS}

In [9]:
startfilter = swap['Date'].isin(ir['Value Date']) 
timefilter = swap['Tenor'].isin(YtoInt.values())
swap_rate = swap[startfilter & timefilter]

swap_rate = swap_rate.drop_duplicates(['Date', 'Tenor'], keep = 'first') # keep the earliest quoted start date of the swap rate
swap_rate

,Date,Start Date,Tenor,Swap Rate
34263,2/9/2022,4/9/2023,10y,2.737709
34267,2/9/2022,4/9/2023,2y,2.915516
34270,2/9/2022,4/9/2023,5y,2.724626
34506,6/9/2022,6/9/2023,10y,2.853063
34510,6/9/2022,6/9/2023,2y,3.026077
...,...,...,...,...
42367,1/2/2023,1/2/2024,2y,2.861414
42370,1/2/2023,1/2/2024,5y,2.670449
42444,2/2/2023,2/2/2024,10y,2.721939
42448,2/2/2023,2/2/2024,2y,2.846523


In [10]:
swap_rate = swap_rate.drop(columns = "Start Date")
swap_rate = swap_rate.rename(columns = {"Tenor": "CMS Tenor", "Date": "Value Date"})
swap_rate

,Value Date,CMS Tenor,Swap Rate
34263,2/9/2022,10y,2.737709
34267,2/9/2022,2y,2.915516
34270,2/9/2022,5y,2.724626
34506,6/9/2022,10y,2.853063
34510,6/9/2022,2y,3.026077
...,...,...,...
42367,1/2/2023,2y,2.861414
42370,1/2/2023,5y,2.670449
42444,2/2/2023,10y,2.721939
42448,2/2/2023,2y,2.846523


In [11]:
ir['CMS Tenor'] = ir['Trade Name'].map(dummyTenorCMS).map(YtoInt)

ir = ir.merge(swap_rate, on = ['Value Date', 'CMS Tenor'], how = "left")
ir

## only map swap rate to zero shock data

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate
0,2/9/2022,dummyTrade1,USD,-100,-227907.0988,1y,4/9/2023,10y,1.962246,2y,2.915516
1,2/9/2022,dummyTrade1,USD,-50,-222208.4010,1y,4/9/2023,10y,-3.812341,2y,2.915516
2,2/9/2022,dummyTrade1,USD,-25,-218960.9280,1y,4/9/2023,10y,4.471006,2y,2.915516
3,2/9/2022,dummyTrade1,USD,-10,-216872.4301,1y,4/9/2023,10y,4.333398,2y,2.915516
4,2/9/2022,dummyTrade1,USD,-5,-216146.3103,1y,4/9/2023,10y,5.679687,2y,2.915516
...,...,...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,-197572.2089,15y,2/2/2038,10y,0.104718,10y,2.721939
1048571,2/2/2023,dummyTrade6,USD,100,-186933.5679,15y,2/2/2038,10y,0.238781,10y,2.721939
1048572,2/2/2023,dummyTrade7,USD,-100,-680981.2765,15y,2/2/2038,10y,2.552405,2y,2.846523
1048573,2/2/2023,dummyTrade7,USD,-50,-656248.2494,15y,2/2/2038,10y,2.687706,2y,2.846523


In [12]:
#ir.to_csv("new ir.csv")

In [13]:
trade = trade.drop(columns = ["underlying", "pay_frequency", "maturity"])
trade.rename(columns={'trade name': 'Trade Name'}, inplace=True)
trade

,Trade Name,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,dummyTrade1,0.0042,0.0379,0.0337,6,5
1,dummyTrade2,0.0042,0.0379,0.0337,4,5
2,dummyTrade3,0.0044,0.0397,0.0353,6,5
3,dummyTrade4,0.0044,0.0397,0.0353,4,5
4,dummyTrade5,0.0046,0.0417,0.0371,4,5
5,dummyTrade6,0.0046,0.0417,0.0371,3,5
6,dummyTrade7,0.0042,0.0379,0.0337,6,10
7,dummyTrade8,0.0042,0.0379,0.0337,4,10
8,dummyTrade9,0.0044,0.0397,0.0353,6,10
9,dummyTrade10,0.0044,0.0397,0.0353,4,10


In [14]:
ir = ir.merge(trade, on = "Trade Name", how = "left")

In [15]:
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,2/9/2022,dummyTrade1,USD,-100,-227907.0988,1y,4/9/2023,10y,1.962246,2y,2.915516,0.0042,0.0379,0.0337,6,5
1,2/9/2022,dummyTrade1,USD,-50,-222208.4010,1y,4/9/2023,10y,-3.812341,2y,2.915516,0.0042,0.0379,0.0337,6,5
2,2/9/2022,dummyTrade1,USD,-25,-218960.9280,1y,4/9/2023,10y,4.471006,2y,2.915516,0.0042,0.0379,0.0337,6,5
3,2/9/2022,dummyTrade1,USD,-10,-216872.4301,1y,4/9/2023,10y,4.333398,2y,2.915516,0.0042,0.0379,0.0337,6,5
4,2/9/2022,dummyTrade1,USD,-5,-216146.3103,1y,4/9/2023,10y,5.679687,2y,2.915516,0.0042,0.0379,0.0337,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,-197572.2089,15y,2/2/2038,10y,0.104718,10y,2.721939,0.0046,0.0417,0.0371,3,5
1048571,2/2/2023,dummyTrade6,USD,100,-186933.5679,15y,2/2/2038,10y,0.238781,10y,2.721939,0.0046,0.0417,0.0371,3,5
1048572,2/2/2023,dummyTrade7,USD,-100,-680981.2765,15y,2/2/2038,10y,2.552405,2y,2.846523,0.0042,0.0379,0.0337,6,10
1048573,2/2/2023,dummyTrade7,USD,-50,-656248.2494,15y,2/2/2038,10y,2.687706,2y,2.846523,0.0042,0.0379,0.0337,6,10


In [16]:
#ir.to_csv("complete ir.csv")
# merged both swap rate and product features data to IR

In [17]:
# feature engineering

In [18]:
ir['Swap Rate'] = ir['Swap Rate']/100

In [19]:
ir.insert(4, "Zero Rate Shock (DEC)", ir['Zero Rate Shock'] / 10000)
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,2/9/2022,dummyTrade1,USD,-100,-0.0100,-227907.0988,1y,4/9/2023,10y,1.962246,2y,0.029155,0.0042,0.0379,0.0337,6,5
1,2/9/2022,dummyTrade1,USD,-50,-0.0050,-222208.4010,1y,4/9/2023,10y,-3.812341,2y,0.029155,0.0042,0.0379,0.0337,6,5
2,2/9/2022,dummyTrade1,USD,-25,-0.0025,-218960.9280,1y,4/9/2023,10y,4.471006,2y,0.029155,0.0042,0.0379,0.0337,6,5
3,2/9/2022,dummyTrade1,USD,-10,-0.0010,-216872.4301,1y,4/9/2023,10y,4.333398,2y,0.029155,0.0042,0.0379,0.0337,6,5
4,2/9/2022,dummyTrade1,USD,-5,-0.0005,-216146.3103,1y,4/9/2023,10y,5.679687,2y,0.029155,0.0042,0.0379,0.0337,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,0.0075,-197572.2089,15y,2/2/2038,10y,0.104718,10y,0.027219,0.0046,0.0417,0.0371,3,5
1048571,2/2/2023,dummyTrade6,USD,100,0.0100,-186933.5679,15y,2/2/2038,10y,0.238781,10y,0.027219,0.0046,0.0417,0.0371,3,5
1048572,2/2/2023,dummyTrade7,USD,-100,-0.0100,-680981.2765,15y,2/2/2038,10y,2.552405,2y,0.028465,0.0042,0.0379,0.0337,6,10
1048573,2/2/2023,dummyTrade7,USD,-50,-0.0050,-656248.2494,15y,2/2/2038,10y,2.687706,2y,0.028465,0.0042,0.0379,0.0337,6,10


In [20]:
ir.insert(12, "Swap Rate (Shocked)", ir['Swap Rate'] + ir['Zero Rate Shock'] / 10000)
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,2/9/2022,dummyTrade1,USD,-100,-0.0100,-227907.0988,1y,4/9/2023,10y,1.962246,2y,0.029155,0.019155,0.0042,0.0379,0.0337,6,5
1,2/9/2022,dummyTrade1,USD,-50,-0.0050,-222208.4010,1y,4/9/2023,10y,-3.812341,2y,0.029155,0.024155,0.0042,0.0379,0.0337,6,5
2,2/9/2022,dummyTrade1,USD,-25,-0.0025,-218960.9280,1y,4/9/2023,10y,4.471006,2y,0.029155,0.026655,0.0042,0.0379,0.0337,6,5
3,2/9/2022,dummyTrade1,USD,-10,-0.0010,-216872.4301,1y,4/9/2023,10y,4.333398,2y,0.029155,0.028155,0.0042,0.0379,0.0337,6,5
4,2/9/2022,dummyTrade1,USD,-5,-0.0005,-216146.3103,1y,4/9/2023,10y,5.679687,2y,0.029155,0.028655,0.0042,0.0379,0.0337,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,0.0075,-197572.2089,15y,2/2/2038,10y,0.104718,10y,0.027219,0.034719,0.0046,0.0417,0.0371,3,5
1048571,2/2/2023,dummyTrade6,USD,100,0.0100,-186933.5679,15y,2/2/2038,10y,0.238781,10y,0.027219,0.037219,0.0046,0.0417,0.0371,3,5
1048572,2/2/2023,dummyTrade7,USD,-100,-0.0100,-680981.2765,15y,2/2/2038,10y,2.552405,2y,0.028465,0.018465,0.0042,0.0379,0.0337,6,10
1048573,2/2/2023,dummyTrade7,USD,-50,-0.0050,-656248.2494,15y,2/2/2038,10y,2.687706,2y,0.028465,0.023465,0.0042,0.0379,0.0337,6,10


In [21]:
ir['Distance_lower (Shocked)'] = ir['Swap Rate (Shocked)'] - ir['lower_bound']
ir['Distance_upper (Shocked)'] = ir['upper_bound'] - ir['Swap Rate (Shocked)']

In [22]:
ir['Out of Bound'] = (ir['Distance_lower (Shocked)'] < 0) |  (ir['Distance_upper (Shocked)'] < 0)
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound
0,2/9/2022,dummyTrade1,USD,-100,-0.0100,-227907.0988,1y,4/9/2023,10y,1.962246,2y,0.029155,0.019155,0.0042,0.0379,0.0337,6,5,0.014955,0.018745,False
1,2/9/2022,dummyTrade1,USD,-50,-0.0050,-222208.4010,1y,4/9/2023,10y,-3.812341,2y,0.029155,0.024155,0.0042,0.0379,0.0337,6,5,0.019955,0.013745,False
2,2/9/2022,dummyTrade1,USD,-25,-0.0025,-218960.9280,1y,4/9/2023,10y,4.471006,2y,0.029155,0.026655,0.0042,0.0379,0.0337,6,5,0.022455,0.011245,False
3,2/9/2022,dummyTrade1,USD,-10,-0.0010,-216872.4301,1y,4/9/2023,10y,4.333398,2y,0.029155,0.028155,0.0042,0.0379,0.0337,6,5,0.023955,0.009745,False
4,2/9/2022,dummyTrade1,USD,-5,-0.0005,-216146.3103,1y,4/9/2023,10y,5.679687,2y,0.029155,0.028655,0.0042,0.0379,0.0337,6,5,0.024455,0.009245,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,0.0075,-197572.2089,15y,2/2/2038,10y,0.104718,10y,0.027219,0.034719,0.0046,0.0417,0.0371,3,5,0.030119,0.006981,False
1048571,2/2/2023,dummyTrade6,USD,100,0.0100,-186933.5679,15y,2/2/2038,10y,0.238781,10y,0.027219,0.037219,0.0046,0.0417,0.0371,3,5,0.032619,0.004481,False
1048572,2/2/2023,dummyTrade7,USD,-100,-0.0100,-680981.2765,15y,2/2/2038,10y,2.552405,2y,0.028465,0.018465,0.0042,0.0379,0.0337,6,10,0.014265,0.019435,False
1048573,2/2/2023,dummyTrade7,USD,-50,-0.0050,-656248.2494,15y,2/2/2038,10y,2.687706,2y,0.028465,0.023465,0.0042,0.0379,0.0337,6,10,0.019265,0.014435,False


In [23]:
ir['Swap_Bound Distance'] = ir[['Distance_lower (Shocked)','Distance_upper (Shocked)']].min(axis=1)
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance
0,2/9/2022,dummyTrade1,USD,-100,-0.0100,-227907.0988,1y,4/9/2023,10y,1.962246,2y,0.029155,0.019155,0.0042,0.0379,0.0337,6,5,0.014955,0.018745,False,0.014955
1,2/9/2022,dummyTrade1,USD,-50,-0.0050,-222208.4010,1y,4/9/2023,10y,-3.812341,2y,0.029155,0.024155,0.0042,0.0379,0.0337,6,5,0.019955,0.013745,False,0.013745
2,2/9/2022,dummyTrade1,USD,-25,-0.0025,-218960.9280,1y,4/9/2023,10y,4.471006,2y,0.029155,0.026655,0.0042,0.0379,0.0337,6,5,0.022455,0.011245,False,0.011245
3,2/9/2022,dummyTrade1,USD,-10,-0.0010,-216872.4301,1y,4/9/2023,10y,4.333398,2y,0.029155,0.028155,0.0042,0.0379,0.0337,6,5,0.023955,0.009745,False,0.009745
4,2/9/2022,dummyTrade1,USD,-5,-0.0005,-216146.3103,1y,4/9/2023,10y,5.679687,2y,0.029155,0.028655,0.0042,0.0379,0.0337,6,5,0.024455,0.009245,False,0.009245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,0.0075,-197572.2089,15y,2/2/2038,10y,0.104718,10y,0.027219,0.034719,0.0046,0.0417,0.0371,3,5,0.030119,0.006981,False,0.006981
1048571,2/2/2023,dummyTrade6,USD,100,0.0100,-186933.5679,15y,2/2/2038,10y,0.238781,10y,0.027219,0.037219,0.0046,0.0417,0.0371,3,5,0.032619,0.004481,False,0.004481
1048572,2/2/2023,dummyTrade7,USD,-100,-0.0100,-680981.2765,15y,2/2/2038,10y,2.552405,2y,0.028465,0.018465,0.0042,0.0379,0.0337,6,10,0.014265,0.019435,False,0.014265
1048573,2/2/2023,dummyTrade7,USD,-50,-0.0050,-656248.2494,15y,2/2/2038,10y,2.687706,2y,0.028465,0.023465,0.0042,0.0379,0.0337,6,10,0.019265,0.014435,False,0.014435
